In [113]:
# Needed Modules

import numpy as np
from astropy.table import Table, vstack
from astropy.io import ascii
import os
import queue
from threading import Thread
import astropy.units as u
import astropy.coordinates as coord

In [299]:
def Pro_file(file_name):
    fh = open(file_name,'r')
    fo = open('data/Gaia_master.csv','a')
    
    header2 = []
    fail = False
    fail_count = 0

    # 2=source_id, 5=ra, 7=dec, 9=parallax , phot_bp_mean_mag=55, rp=60, g = 50
    useful = (0,5,6,7,8,9,10,55,60,50)
    #makes truncated header
    for i in useful:
        header2.append(header1[i])
    #do required calculations by line
    print("start")
    for line in fh:
    
        

        if line != header:
            #turns line string into a list
            line = line.split(",")

        
            #makes truncated line
            line2 = []
            for i in useful:
                if line[i] != '':
                    line2.append(line[i])
                else:#skips line with missing values
                    fail = True
                    continue
                
            if fail == True:
                fail = False
                fail_count = fail_count + 1
                
                continue
        


        
            #convert to galactocentric

            ra = float(line2[1])
            dec = float(line2[3])
            dist = 1/(float(line2[5])*10**(-3))
            
            c = coord.ICRS(ra=ra * u.degree, dec=dec * u.degree, distance=dist * u.pc)
        
            g = c.transform_to(coord.Galactocentric)
        
            #galactocentric coords are x,y,z, convert to r,theta,phi (spherical) [theta= angle from z axis, phi = angle from x axis]
            g_r = np.sqrt((g.x)**2 + (g.y)**2 + (g.z)**2)
            gtheta = np.degrees(np.arctan((np.sqrt(g.x**2 +g.y**2))/g.z))
            gphi = np.degrees(np.arctan(g.y/g.x))
            #print(gr , gtheta, gphi)
        
        
            #calculate colors
            br = float(line2[7])-float(line2[8]) #blue-red
            bg = float(line2[7])-float(line2[9]) #blue-green
            gr = float(line2[9])-float(line2[8]) #green-red
        
        
        
            #convert values to strings
            ident = str(line2[0])
            g_r = str(g_r.value)
            gtheta = str(gtheta.value)
            gphi = str(gphi.value)
        
            br = str(br)
            bg = str(bg)
            gr = str(gr)
        
            #write string to new file
            fo.write(ident)
            fo.write(",")
            fo.write(g_r)
            fo.write(",")
            fo.write(gtheta)
            fo.write(",")
            fo.write(gphi)
            fo.write(",")
        
            fo.write(br)
            fo.write(",")
            fo.write(bg)
            fo.write(",")
            fo.write(gr)
        
        
            fo.write("\n")
            #print(ident,gr,gtheta,gphi)
    print(fail_count, "failures")   
    print("done")
    fo.close()
      
    
    
    

In [300]:
###Running this creates new database and inserts header###
###WARNING if database already exists this will over write###
fo = open('data/Gaia_master.csv','w')
fo.write("Solution_ID,R,Theta,Phi,BR,BG,GR\n")
fo.close()


#header line on all input data files
header1 = ['solution_id', 'designation', 'source_id', 'random_index', 'ref_epoch', 'ra', 'ra_error', 'dec', 'dec_error', 'parallax', 'parallax_error', 'parallax_over_error', 'pmra', 'pmra_error', 'pmdec', 'pmdec_error', 'ra_dec_corr', 'ra_parallax_corr', 'ra_pmra_corr', 'ra_pmdec_corr', 'dec_parallax_corr', 'dec_pmra_corr', 'dec_pmdec_corr', 'parallax_pmra_corr', 'parallax_pmdec_corr', 'pmra_pmdec_corr', 'astrometric_n_obs_al', 'astrometric_n_obs_ac', 'astrometric_n_good_obs_al', 'astrometric_n_bad_obs_al', 'astrometric_gof_al', 'astrometric_chi2_al', 'astrometric_excess_noise', 'astrometric_excess_noise_sig', 'astrometric_params_solved', 'astrometric_primary_flag', 'astrometric_weight_al', 'astrometric_pseudo_colour', 'astrometric_pseudo_colour_error', 'mean_varpi_factor_al', 'astrometric_matched_observations', 'visibility_periods_used', 'astrometric_sigma5d_max', 'frame_rotator_object_type', 'matched_observations', 'duplicated_source', 'phot_g_n_obs', 'phot_g_mean_flux', 'phot_g_mean_flux_error', 'phot_g_mean_flux_over_error', 'phot_g_mean_mag', 'phot_bp_n_obs', 'phot_bp_mean_flux', 'phot_bp_mean_flux_error', 'phot_bp_mean_flux_over_error', 'phot_bp_mean_mag', 'phot_rp_n_obs', 'phot_rp_mean_flux', 'phot_rp_mean_flux_error', 'phot_rp_mean_flux_over_error', 'phot_rp_mean_mag', 'phot_bp_rp_excess_factor', 'phot_proc_mode', 'bp_rp', 'bp_g', 'g_rp', 'radial_velocity', 'radial_velocity_error', 'rv_nb_transits', 'rv_template_teff', 'rv_template_logg', 'rv_template_fe_h', 'phot_variable_flag', 'l', 'b', 'ecl_lon', 'ecl_lat', 'priam_flags', 'teff_val', 'teff_percentile_lower', 'teff_percentile_upper', 'a_g_val', 'a_g_percentile_lower', 'a_g_percentile_upper', 'e_bp_min_rp_val', 'e_bp_min_rp_percentile_lower', 'e_bp_min_rp_percentile_upper', 'flame_flags', 'radius_val', 'radius_percentile_lower', 'radius_percentile_upper', 'lum_val', 'lum_percentile_lower', 'lum_percentile_upper\n']

files = [
    "data/GaiaSource_1584380076484244352_2200921635402776448.csv", 
    "data/GaiaSource_2200921875920933120_3650804325670415744.csv",
    "data/GaiaSource_2851858288640_1584379458008952960.csv", 
    "data/GaiaSource_3650805523966057472_4475721411269270528.csv", 
    "data/GaiaSource_4475722064104327936_5502601461277677696.csv", 
    "data/GaiaSource_5502601873595430784_5933051501826387072.csv", 
    "data/GaiaSource_5933051914143228928_6714230117939284352.csv", 
    "data/GaiaSource_6714230465835878784_6917528443525529728.csv"]



In [ ]:
##This appends calculations to end of database##
###WARNING running this if database already exists will add duplicate entries###

for i in files:
    Pro_file(i)

print("done")

start
